In [1]:
from ultralytics import YOLO
import Library.package_face_recognition as fr
import cv2
import cv2.face
import numpy as np



# Load the model YOLO
model_YOLO = YOLO("Library/yolov8m.pt")

# Load the model LBPH

model_lbph = cv2.face.LBPHFaceRecognizer_create()
model_lbph.read("Library/lbph_trained_model.yml")


# Load the image

image = cv2.imread("Images/face_recognition.jpg")

# Load the label dictionary
label_dict = np.load('Library/label_dict.npy', allow_pickle=True).item()

# Detect the objects
results = model_YOLO.predict(image)

result = results[0]
box = result.boxes[0]


# if object is a person then crop the image and detect the face
for i in range(len(result.boxes)):
    box = result.boxes[i]
    cords = box.xyxy[0].tolist()
    cords = [round(x) for x in cords]
    class_id = result.names[box.cls[0].item()]
    conf = round(box.conf[0].item(), 2)

    
    if class_id == 'person':
        print ("person find : ",cords)
        cropped_image = fr.cut_image(image, cords)
        cv2.imwrite(f"image_copped1{cords}.jpg", cropped_image)

        cropped_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB)
        
        # cv2.imshow('cropped_image', cropped_image)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()

        try:
            label, coordinates = fr.face_recognition(cropped_image, model_lbph)

            # find the corresponding name for the integer label
            for name, l in label_dict.items():
                if l == label:
                    print(name)
                    break
            cords = coordinates + cords
                    
                    
        except:
            name= class_id
            print("No face detected 2")

        
  
        
        print (cords)
        image = fr.boundary_Name_Box(image, name, cords)
        # cv2.imshow('image', image)


from PIL import Image
Image.fromarray(result.plot()[:,:,::-1]).save('image.jpg')

cv2.imwrite("image_label.jpg", image)


error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv_contrib\modules\face\src\facerec.cpp:61: error: (-2:Unspecified error) File can't be opened for reading! in function 'cv::face::FaceRecognizer::read'


In [1]:
import cv2
print(cv2.__version__)

4.8.1


In [2]:
import torch
print(torch.__version__)

2.1.0+cpu
